In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload
from trackingutil import within_bound, get_bound, object_tracker, video_tracker

In [2]:
import unittest
import time
class TestUtilMethods(unittest.TestCase):

    def test_within_bound(self):
        self.assertTrue(within_bound(((1,2),(4,5)),((2,3),(4,4))))
        self.assertTrue(within_bound(((1,2),(4,5)),((1,3),(4,5))))
        self.assertFalse(within_bound(((1,2),(4,5)),((0,3),(4,5))))
        self.assertFalse(within_bound(((1,2),(4,5)),((1,3),(5,5))))
    
    def test_get_bound(self):
        self.assertEqual(((9, 9), (16,16)), get_bound(((10, 10), (15,15)), 0.1))
        self.assertEqual(((9, 9), (16,16)), get_bound(((10, 10), (15,15)), 0.2))
        self.assertEqual(((9, 9), (16,16)), get_bound(((10, 10), (15,15)), 0.4))
        self.assertEqual(((8, 8), (17,17)), get_bound(((10, 10), (15,15)), 0.8))
    def test_object_tracker(self):
        testTracker = object_tracker(0.5, 0.1)
        self.assertTrue(testTracker.Inframe)
        self.assertEqual(None,testTracker.bound)
        self.assertTrue(testTracker.track(((1,2),(3,4))))
        self.assertEqual(get_bound(((1,2),(3,4)), 0.1),testTracker.bound)
        self.assertTrue(testTracker.Inframe)
        self.assertFalse(testTracker.track(((-1,2),(3,4))))
        self.assertEqual(get_bound(((1,2),(3,4)), 0.1),testTracker.bound)
        time.sleep(1)
        self.assertFalse(testTracker.track(((-1,2),(3,4))))
        self.assertFalse(testTracker.Inframe)
        self.assertTrue(1 > testTracker.timeInframe())
        self.assertTrue(testTracker.timeInframe() >= 0.5)
        testTracker = object_tracker(0.5, 0.1)
        self.assertEqual(None,testTracker.bound)
        self.assertTrue(testTracker.track(((1,2),(3,4))))
        self.assertEqual(get_bound(((1,2),(3,4)), 0.1),testTracker.bound)
        self.assertFalse(testTracker.track(((-1,2),(3,4))))
        self.assertEqual(get_bound(((1,2),(3,4)), 0.1),testTracker.bound)
        time.sleep(1)
        self.assertFalse(testTracker.track(None))
        self.assertTrue(1 > testTracker.timeInframe())
        self.assertTrue(testTracker.timeInframe() >= 0.5)
    def test_video_tracker(self):
        vidTracker = video_tracker(0.5, 0.1)
        vidTracker.track(((1,2),(3,4)))
        self.assertEqual(1, len(vidTracker.active_obs_trackers))
        time.sleep(1)
        vidTracker.track(None)
        self.assertEqual(0, len(vidTracker.active_obs_trackers))
        self.assertEqual(1, len(vidTracker.oof_obs_trackers))
        vidTracker.track(((1,2),(3,4)))
        self.assertEqual(1, len(vidTracker.active_obs_trackers))
        time.sleep(1)
        vidTracker.track(((-1,2),(3,4)))
        self.assertEqual(1, len(vidTracker.active_obs_trackers))
        self.assertEqual(2, len(vidTracker.oof_obs_trackers))
        newObs = object_tracker(0, 0)
        newObs2 = object_tracker(0, 0)
        newObs.lastInframe = 2
        newObs.firstInframe = 0
        newObs2.lastInframe = 1
        newObs2.firstInframe = 0
        vidTracker.active_obs_trackers = [newObs2]
        vidTracker.oof_obs_trackers = [newObs]
        self.assertEqual(1.5, vidTracker.get_average())
        self.assertEqual(2, vidTracker.get_num_objects())
        self.assertEqual(1, vidTracker.get_num_current_objects())
TestUtilMethods().test_within_bound()
TestUtilMethods().test_get_bound()
TestUtilMethods().test_object_tracker()
TestUtilMethods().test_video_tracker()